In [ ]:
def get_lgbm(x_tr, y_tr, x_val, y_val, shape):
    y_valid = y_val
    y_tr = np.argmax(y_tr, axis=1)
    y_val = np.argmax(y_val, axis=1)
    trn_data = lgb.Dataset(x_tr, label=y_tr)
    val_data = lgb.Dataset(x_val, label=y_val)
    model = lgb.train(param, trn_data, 10000, valid_sets = [val_data], verbose_eval = 100, early_stopping_rounds = 200)
    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    crps = crps_score(y_pred, y_valid, shape=shape)
    return model, crps
# Calculate CRPS score
def crps_score(y_prediction, y_valid, shape=X.shape[0]):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

import lightgbm as lgb

metric = "multi_logloss"
param = {'num_leaves': 50,
         'min_data_in_leaf': 30,
         'objective':'multiclass',
         'num_class': 199,
         'max_depth': 6, # -1
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.4, #0.7
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": metric,
         "lambda_l1": 0.1,
         "verbosity": -1,
         'n_jobs': -1,
         "seed":1234}
y = get_y(yards)

loop = 2
fold = 5

oof_nn = np.zeros([loop, y.shape[0], y.shape[1]])
oof_lgbm = np.zeros([loop, y.shape[0], y.shape[1]])
oof_rf = np.zeros([loop, y.shape[0], y.shape[1]])

models_nn = []
crps_csv_nn = []

models_lgbm = []
crps_csv_lgbm = []

models_rf = []
crps_csv_rf = []

feature_importance = np.zeros([loop, fold, X.shape[1]])

s_time = time.time()

for k in range(loop):
    kfold = KFold(fold, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x, tr_y = X[tr_inds], y[tr_inds]
        val_x, val_y = X[val_inds], y[val_inds]
        
        
        # Train LGBM
        lgbm, crps_lgbm = get_lgbm(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        models_lgbm.append(lgbm)
        print("the %d fold crps (LGBM) is %f"%((k_fold+1), crps_lgbm))
        crps_csv_lgbm.append(crps_lgbm)
        
#         # Train RF
#         rf, crps_rf = get_rf(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
#         models_rf.append(rf)
#         print("the %d fold crps (RF) is %f"%((k_fold+1), crps_rf))
#         crps_csv_rf.append(crps_rf)
        
        # Feature Importance
        feature_importance[k, k_fold, :] = lgbm.feature_importance()
#         feature_importance[k, k_fold, :] = rf.feature_importances_
        
        #Predict OOF
        oof_lgbm[k, val_inds, :] = lgbm.predict(val_x, num_iteration=lgbm.best_iteration)
#         oof_rf[k, val_inds, :] = rf.predict(val_x)

feature_importances = pd.DataFrame(np.mean(feature_importance, axis=0).T, columns=[['fold_{}'.format(fold_n) for fold_n in range(fold)]])
feature_importances['feature'] = input_df.columns
feature_importances['average'] = feature_importances[['fold_{}'.format(fold_n) for fold_n in range(fold)]].mean(axis=1)
feature_importance_flatten = pd.DataFrame()
for i in range(len(feature_importances.columns)-2):
    col = ['feature', feature_importances.columns.values[i][0]]
    feature_importance_flatten = pd.concat([feature_importance_flatten, feature_importances[col].rename(columns={'fold_{}'.format(i) :
                                                                                                                 'importance'})], axis=0)

plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importance_flatten.sort_values(by=('importance',), ascending=False), x=('importance',), y=('feature',))
plt.title('Feature Importances over {} loops and {} folds'.format(loop, fold))
plt.savefig("feature_importance.png")
plt.show()
